In [39]:
import argparse
import gym
import gymfc
import numpy as np
from mpi4py import MPI
from collections import OrderedDict
import math
import os
import time


import nest_asyncio
nest_asyncio.apply()

In [40]:
current_dir = os.getcwd()
print(current_dir)
config_path = os.path.join(current_dir,
                               "../configs/iris.config")
print(config_path)
os.environ["GYMFC_CONFIG"] = config_path

/home/ec2-user/nti/experiments_bezvershenko/controllers
/home/ec2-user/nti/experiments_bezvershenko/controllers/../configs/iris.config


In [41]:
# Код агента
from pid import Agent

In [65]:
def evaluate(env, agent, seed, model_name, verbose):
    actuals = []
    desireds = []
    agent.reset()
    ob = env.reset()
    ticks = 0
    statistic = [model_name, seed]
    sum_rewards = 0
    
    while True:
        desired = env.omega_target
        actual = env.omega_actual

        ac = agent.predict(ob, env.sim_time, desired, actual)

        ob, reward, done, info = env.step(ac)
        
       
        
        actuals.append(actual)
        desireds.append(desired)
        ticks += 1
        sum_rewards += reward
        
        if ticks % 100 == 0 or ticks == 1:
            if verbose:
                print()
                print('Tick #'+str(ticks))
                print('='*20)
                print('Need:', desired)
                print('Right now:', actual)
                print('Is done:', done)
                print('Reward:', reward)
                print('='*20)
                print()
            statistic.append(reward)
        
        if done:
            statistic.append(sum_rewards)
            statistic.append(sum_rewards / 1000)
            break
            
    env.close()
    return desireds, actuals, list(map(str, statistic))

In [66]:
def main(env_id='AttFC_GyroErr-MotorVel_M4_Ep-v0', verbose=False, seed=17, model_name='pid', writing_mode=False, w_file='statistic.csv'):
    env = gym.make(env_id)
    
    rank = MPI.COMM_WORLD.Get_rank()
    workerseed = seed + 1000000 * rank
    env.seed(workerseed)
    
    agent = Agent()
    desireds, actuals, statistic = evaluate(env, agent, seed, model_name, verbose)    
    
    if writing_mode:
        with open(w_file, 'a') as file:
            file.write(';'.join(statistic)+'\n')

In [67]:
for s in range(1, 50):
    main(seed=s, writing_mode=True)

Starting gzserver with process ID= 16352

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085163.0497756
time_lapse_hours       0.001466974549823337


Killing process with ID= 16352
Starting gzserver with process ID= 16401

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085168.5885315
time_lapse_hours       0.0013852715492248535


Killing process with ID= 16401
Starting gzserver with process ID= 16450

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085173.8261712
time_lapse_hours       0.0021307315429051716


Killing process with ID= 16450
Starting gzserver with process ID= 16499

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085181.8188143
time_lapse_hours       0.0015549092160330878


Killing process with ID= 16499
S

Starting gzserver with process ID= 19060

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085367.190698
time_lapse_hours       0.0015310929218928019


Killing process with ID= 19060
Starting gzserver with process ID= 19257

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085373.1690419
time_lapse_hours       0.0015060248639848497


Killing process with ID= 19257
Starting gzserver with process ID= 19397

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085378.8093238
time_lapse_hours       0.0017070321242014567


Killing process with ID= 19397
Starting gzserver with process ID= 19448

Simulation Stats
-----------------
steps                  1001
packets_dropped        0
time_start_seconds     1553085385.2219756
time_lapse_hours       0.0016236324442757502


Killing process with ID= 19448
S